# Trabalho 1 — Mat. Comp. 2 — 2024.1

## Identificação dos autores

* Estudante 1:
    * _Nome:_
    * _Matrícula:_
* Estudante 2 (se houver):
    * _Nome:_
    * _Matrícula:_
* Estudante 3 (se houver):
    * _Nome:_
    * _Matrícula:_

## Instruções

* O trabalho pode ser resolvido por um grupo de no máximo 3 alunos.
* Utilize o Google Colab para editar este arquivo (exatamente como nas aulas). Os enunciados das questões estão na próxima seção.
* Preencha a identificação de cada um(a) acima. Para isto:
  * Clique duas vezes com o botão esquerdo do mouse na célula acima para entrar no modo de edição.
  * Preencha os dados sem modificar a parte que já está escrita.
  * Após terminar, aperte `Shift + Enter` para sair do modo de edição e conferir o resultado.
* Resolva os problemas enunciados abaixo utilizando apenas células de código.
  * Para inserir uma nova célula de código, selecione uma célula qualquer clicando na região à esquerda dela e então pressionando `b` ou `a` para criar uma nova célula abaixo ou acima, respectivamente.
  * Caso deseje explicar melhor algum passo, pode utilizar comentários Python (usando `#`).
* _O envio deve ser feito até quarta-feira dia 10/07 às 23:59:59_. Trabalhos recebidos depois deste horário serão desconsiderados.
* Cada item vale $ 2 $ pontos, para um total de $ 10 $ pontos.

## Problemas

__Problema 1 ([cifra de Hill](https://en.wikipedia.org/wiki/Hill_cipher)):__
Neste problema usaremos um método simples baseado em álgebra linear para
codificar e decodificar mensagens.  O algoritmo funciona da seguinte maneira.

* Atribua um valor numérico aos caracteres
$
    \big(\text{espaço} = 0,\ \ A = 1,\ \ B = 2,\ \cdots,\ \  Z = 26,\ \ \text{ponto} = 27
    , \ \ \backslash \texttt{n} = 28 \big).
$
É importante que o número total de símbolos (no caso $ 29 $) seja primo.

* Escolha uma mensagem com número par de caracteres para codificar. Represente-a 
como uma matriz $ 2 \times n $, onde cada entrada corresponde a uma letra. Por
exemplo, "PYTHON" seria representado como
$$
\begin{bmatrix}
P & Y & T \\
H & O & N
\end{bmatrix}
\longleftrightarrow
\begin{bmatrix}
16 & 25 & 20 \\
8 & 15 & 14
\end{bmatrix}\,.
$$

* Denote por $ \mathbb{Z}_{29} $ o conjunto dos números entre $ 0 $ e $ 28 $ com
  as operações $ \oplus $ e $ \odot $ de adição e multiplicação módulo
  $ 29 $. Ou seja, para $ a, b \in \mathbb{Z}_{29} $:
  - **adição**: $ a \oplus b $ é calculada como a soma usual de $ a $ e $ b $,
    seguida da redução módulo $ 29 $. Formalmente, $ a \oplus b = a + b \pmod{29} $. Por exemplo,
    $ 17 \oplus 18 = 35 \pmod{29} = 6 $, porque o resto da divisão inteira de $
    35 $ por $ 29 $ é $ 6 $.
  - **multiplicação**: $ a \odot b $ é calculado como o produto usual de $ a $ e
    $ b $, seguido da redução módulo $ 29 $. Formalmente, $ a \odot b = a \cdot
    b \pmod{29} $. Por exemplo, $ 17 \odot 18 = 306 \pmod{29} = 16 $, porque o
    resto da divisão de $ 306 $ por $ 29 $ é $ 16 $.
  
  Estas operações têm propriedades análogas à adição e multiplicação usuais (são
  comutativas, associativas, vale a distributividade, etc.). Em particular, o
  elemento neutro da adição é $ 0 $ e o da multiplicação é $ 1 $.

* Escolha uma matriz
  $$
  C = \begin{bmatrix}
  a & b \\
  c & d 
  \end{bmatrix}
  $$
  qualquer com entradas em $ \mathbb Z_{29} $, desde que ela seja invertível como
  matriz sobre $ \mathbb Z_{29} $, ou seja, desde que o seu determinante $ ad -
  bc $ _não_ seja múltiplo de $ 29 $.  A matriz $ C $ é a chave para o
  processo de codificação. Mais precisamente, para codificar uma mensagem como
  "PYTHON", primeiro a convertemos numa matriz $ M $ de forma $ 2\times n $
  como explicado acima, depois tomamos o produto  $ CM $ usual, reduzimos todas
  as entradas do resultado módulo $ 29 $ e finalmente convertemos este último na
  mensagem correspondente, de novo pela equivalência discutida acima.

* A chave para decodificação é a
  inversa $ C^{-1} $ de $ C $ _como matriz sobre_ $ \mathbb Z_{29} $ (é explicado
  abaixo como calculá-la). Ou seja, seguimos o mesmo procedimento que para a
  codificação, mas com $ C^{-1} $ no lugar de $ C $.


Funções auxiliares `mensagem_para_matriz` e `matriz_para_mensagem` que convertem uma
mensagem (string) de comprimento par na matriz $ 2\times n $ sobre $ \mathbb Z_{29} $
correspondente, e reciprocamente, conforme descrito acima, já estão fornecidas abaixo.
Recorde também que `m % n` é o resto da divisão de $ m $ por $ n $; você irá
precisar deste operador em diversos momentos na resolução dos itens.

In [4]:
import numpy as np

# O seguinte dicionário traduz caracteres para números:
char_to_num = {
    ' ': 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8, 'I': 9,
    'J': 10, 'K': 11, 'L': 12, 'M': 13, 'N': 14, 'O': 15, 'P': 16, 'Q': 17, 'R': 18,
    'S': 19, 'T': 20, 'U': 21, 'V': 22, 'W': 23, 'X': 24, 'Y': 25, 'Z': 26, '.': 27,
    '\n': 28
}
# E este é o dicionário de mapeamento de números para caracteres:
num_to_char = {v: k for k, v in char_to_num.items()}


def mensagem_para_matriz(mensagem):
    """Converte uma mensagem em uma matriz 2 x n de números em Z_29."""
    # Verificando se o número de caracteres é par:
    if len(mensagem) % 2 != 0:
        raise ValueError("O número de caracteres na mensagem deve ser par!")
    
    # Convertendo cada caracter ao seu correspondente numérico:
    nums = [char_to_num[char] for char in mensagem]
    
    # Convertendo a lista acima em uma matriz 2 x n e retornando esta matriz:
    matriz = np.array(nums).reshape(2, -1)
    return matriz


def matriz_para_mensagem(matriz):
    """Converte uma matriz 2 x n de números em Z_29 em uma mensagem (string)."""
    # Convertendo a matriz numa lista de números:
    nums = matriz.flatten()
    
    # Convertendo cada número ao seu caracter correspondente:
    mensagem = ''.join(num_to_char[num] for num in nums)
    return mensagem

In [5]:
# Exemplo:
mensagem = "PYTHON"
matriz = mensagem_para_matriz(mensagem)
print("Matriz correspondente à mensagem \"PYTHON\":")
print(matriz, '\n')

mensagem = matriz_para_mensagem(matriz)
print("Mensagem correspondente a esta matriz:", mensagem)

Matriz correspondente à mensagem "PYTHON":
[[16 25 20]
 [ 8 15 14]] 

Mensagem correspondente a esta matriz: PYTHON


Agora resolva os seguintes itens:

(a) Qualquer $ a \in \mathbb Z_{29} $ diferente de $ 0 $ tem um inverso
multiplicativo $ a^{-1} $. Ele é o único elemento $ b $ de $ \mathbb Z_{29} $
tal que $ a\odot b = b \odot a = 1 $. Crie um procedimento
`inverso(a)` em Python que recebe um número $ a $ entre $ 1 $ e $ 28 $ e
calcula o seu inverso multiplicativo em $ \mathbb Z_{29} $.  Verifique que o
inverso de $ 17 $ é $ 12
$.  _Dica:_ Use um laço de repetição `for` para testar todas as possibilidades
até encontrar o inverso.


In [ ]:
# Item (a)

(b) A inversa sobre $ \mathbb Z_{29} $ de uma matriz $ C = \begin{bmatrix}a & b \\ c & d \end{bmatrix} $
com entradas em $ \mathbb Z_{29} $ é dada por
$$
C^{-1} = \det(C)^{-1} \begin{bmatrix}
d & -b \\
-c & a
\end{bmatrix} \pmod{29}\,.
$$
Ou seja, a expressão é idêntica àquela para a inversa de uma matriz $ 2\times 2 $ de entradas
reais, exceto que o inverso de $ \det(C) $ é tomado em $ \mathbb Z_{29} $ e ao
final precisamos reduzir os coeficientes módulo $ 29 $. Crie um procedimento `matriz_inversa(C)`
que toma uma matriz $ 2 \times 2 $ invertível $ C $ com entradas em $ \mathbb Z_{29} $ e
retorna sua inversa. Utilize arrays NumPy para representar as matrizes. Para testar seu
procedimento, verifique que a inversa de 
$$
C = \begin{bmatrix}
1 & 2 \\
3 & 4
\end{bmatrix} \quad \text{é} \quad
C^{-1} = \begin{bmatrix}
27 & 1 \\
16 & 14
\end{bmatrix}\,.
$$
_Dica:_ Aproveite o procedimento `inverso` do item (a).

In [2]:
# Item (b)

(c) Crie um procedimento `codificar(C, mensagem)` que codifica uma mensagem
arbitrária com número par de caracteres usando a matriz-chave $ C $, seguindo
o algoritmo descrito no início. Verifique que se $ D $ é a inversa de $ C $,
então `codificar(D, codificar(C, mensagem))` é a mensagem original.
_Dica:_ Primeiro crie um procedimento para multiplicar dois arrays compatíveis
sobre $ \mathbb Z_{29} $.  Utilize as funções auxiliares fornecidas anteriormente.

In [3]:
# Item (c)

__Problema 2 (integração sobre um tetraedro):__ Considere o tetraedro $ T $ de vértices 
$$
(0, 0, 0), \quad (1, 0, 0), \quad (0, 1, 0), \quad (0, 0, 1)\,.
$$
Utilizando o SymPy:

(a) Calcule o volume de $ T $.
_Dica:_ Desenhe uma figura à mão para encontrar os limites de integração.

In [2]:
# Item (a)

(b) Calcule a terceira coordenada $ \bar z $ do centróide de $ T $, definida por
$$
\bar z\, \iiint_T\, 1\,dx\,dy\,dz = \iiint_T\, z\,dx\,dy\,dz\,.
$$
Informalmente, $ \bar z $ é a altura média dos pontos de $ T $.

In [3]:
# Item (b)